<a href="https://colab.research.google.com/github/iranfromiran/nlp-assignments/blob/main/sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sentiment analysis of revires of a product called Alexa on Amazon

In [66]:
myDictionary ={}
for roww in open('/content/AFINN-111.txt'):
    word, score = roww.split('\t')
    myDictionary[word] = int(score)

In [67]:
import nltk
import nltk.classify.util #calculates accuracy
from nltk.classify import NaiveBayesClassifier #imports the classifier Naive Bayes
from nltk.corpus import movie_reviews #imports movie reviews from nltk
from nltk.corpus import stopwords #imports stopwords from nltk
from nltk.corpus import wordnet #imports wordnet(lexical database for the english language) from nltk

In [68]:
import pandas as pd
dataSet_path = '/content/amazon_alexa.tsv'
df = pd.read_csv(dataSet_path, sep='\t')

# Extract the 'verified_reviews' and 'feedback' columns cause thta's what we're gonna be using
verified_reviews = df['verified_reviews']
feedback = df['feedback']
verified_reviews_words = df['verified_reviews'].str.split().explode()

# Print the first few rows of each column to see if we did it right or not
print("Verified Reviews:")
print(verified_reviews.head())
print("\nFeedback:")
print(feedback.head())

Verified Reviews:
0                                        Love my Echo!
1                                            Loved it!
2    Sometimes while playing a game, you can answer...
3    I have had a lot of fun with this thing. My 4 ...
4                                                Music
Name: verified_reviews, dtype: object

Feedback:
0    1
1    1
2    1
3    1
4    1
Name: feedback, dtype: int64


In [69]:
#frequency distribution of words in reviews
freq_dist = nltk.FreqDist(verified_reviews_words)
freq_dist.most_common(10)

[('the', 2879),
 ('I', 2835),
 ('to', 2772),
 ('and', 2169),
 ('it', 1673),
 ('a', 1458),
 ('my', 1278),
 ('is', 1188),
 ('for', 1014),
 ('with', 768)]

In [70]:
import nltk
nltk.download('stopwords')
# clearing the input
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words]) #so the words are not repeated
    return my_dict


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
neg_reviews = []

for index, row in df.iterrows():
    review_text = row['verified_reviews']
    feedback = row['feedback']  # 'feedback' column contains 0 or 1
    # Check if the feedback is negative or 0
    if feedback == 0:
        # Tokenize the review text (split into words)
        if isinstance(review_text, str):
          words = review_text.split()

        neg_reviews.append((create_word_features(words), "negative"))
print(len(neg_reviews))

257


In [72]:
pos_reviews = []

for index, row in df.iterrows():
    review_text = row['verified_reviews']
    feedback = row['feedback']  # 'feedback' column contains 0 or 1
    # Check if the feedback is positive or 1
    if feedback == 1:
        # Tokenize the review text (split into words)
        if isinstance(review_text, str):
          words = review_text.split()

        pos_reviews.append((create_word_features(words), "positive"))
print(len(pos_reviews))

2893


In [73]:
train_set = neg_reviews[:192] + pos_reviews[:17170] #75% of the data
test_set =  neg_reviews[192:] + pos_reviews[17170:]
print(len(train_set),  len(test_set))

3085 65


In [74]:
#create the NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(10)

Most Informative Features
                  return = True           negati : positi =     49.3 : 1.0
                     act = True           negati : positi =     45.0 : 1.0
                 either. = True           negati : positi =     45.0 : 1.0
                 Instead = True           negati : positi =     35.0 : 1.0
                   Maybe = True           negati : positi =     35.0 : 1.0
               defective = True           negati : positi =     35.0 : 1.0
                  rarely = True           negati : positi =     35.0 : 1.0
                  review = True           negati : positi =     35.0 : 1.0
                 seconds = True           negati : positi =     35.0 : 1.0
                 useless = True           negati : positi =     35.0 : 1.0


In [75]:
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy * 100)

83.07692307692308


In [76]:
review_1 = '''that's practical'''
review_2 = '''it dosen't work for me and the packaging was bad'''
review_3 = '''i can cook and listen to music and skip my playlist'''

In [77]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
words1 = word_tokenize(review_1)
words1 = create_word_features(words1)
print(classifier.classify(words1)
)
words2 = word_tokenize(review_2)
words2 = create_word_features(words2)
print(classifier.classify(words2))

words3 = word_tokenize(review_3)
words3 = create_word_features(words3)
print(classifier.classify(words3))

positive
negative
positive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
